# Pre-process HDB Data
This notebook loads the 4 datasets and combines them into one master dataset for use later. There are some transformations done to the columns of the data like combining columns to form a proper address and using OneMap APIs (Application Programming Interfaces) to find the associated postal codes (missing from HDB data). These postal codes will be used mostly during visualization. Rows that contain missing values are also dropped.

Load the libraries needed to run the notebook.

In [ ]:
import numpy as np
import pandas as pd
import requests
import tqdm

Specify the folder where the data is stored.

In [ ]:
DATA_DIR = './Data/'

### Column Transformations
Load the 4 csv files downloaded from [here](https://data.gov.sg/dataset/resale-flat-prices) into Pandas dataframes. Compare their column headers to make sure all of the same column names. Load the first 5 rows to understand the data type and to ensure that they are all the same across the datasets. By the way, one of the files has an extra column compared to the other 3.

In [ ]:
# Read CSV files in one-by-one and inspect columns
df1 = pd.read_csv(DATA_DIR+'resale-flat-prices-based-on-approval-date-1990-1999.csv')
df1.columns

In [ ]:
# See the first 5 rows
df1.head()

In [ ]:
# Read CSV files in one-by-one and inspect columns
df2 = pd.read_csv(DATA_DIR+'resale-flat-prices-based-on-approval-date-2000-feb-2012.csv')
df2.columns

In [ ]:
# See the first 5 rows
df2.head()

In [ ]:
# Read CSV files in one-by-one and inspect columns
df3 = pd.read_csv(DATA_DIR+'resale-flat-prices-based-on-registration-date-from-mar-2012-to-dec-2014.csv')
df3.columns

In [ ]:
# See the first 5 rows
df3.head()

In [ ]:
# Read CSV files in one-by-one and inspect columns
df4 = pd.read_csv(DATA_DIR+'resale-flat-prices-based-on-registration-date-from-jan-2015-onwards.csv')
df4.columns

In [ ]:
# See the first 5 rows
df4.head()

#### Create missing column

df4 contains an extra column compared to the other 3 - "remaining_lease". Pandas merging only works if all the columns of the dataframes have the same headers and the number of columns in all dataframes are the same. What is the meaning of this column? HDB flats are sold on a 99-year lease to the general public. What this means is that the value of the property at the end of the lease is deemed to be $0 and the government will take back the flats at the end of the lease for redevelopment purposes. As such, the number of years remaining on the lease agreement at the point of sale does affect the prices of HDB flats to a certain extent. Hence, it will be wise to keep this column for now instead of dropping it. 

However, this means that we need to create the same column in the other 3 dataframes so that they all have the same number and type of columns. Now, this column can be easily created in the other dataframes. There is a column which indicates the year in which the lease commenced - "lease_commence_date" and the year in which the flat was sold - "month". The " "remaining_lease" can then be computed as follows:- 99 - ("month"[year] - lease_commence_date).

In [ ]:
# But first, convert the datetime columns in all the dataframes to datetime format. At the moment, they are of type string
df1['month'] = pd.to_datetime(df1['month'], format="%Y-%m")
df1['lease_commence_date'] = pd.to_datetime(df1['lease_commence_date'], format="%Y")

In [ ]:
## To do
## Repeat the steps from the cell above for the remaining dataframes


**Solution:**

In [ ]:
# Solution
df2['month'] = pd.to_datetime(df2['month'], format="%Y-%m")
df2['lease_commence_date'] = pd.to_datetime(df2['lease_commence_date'], format="%Y")
df3['month'] = pd.to_datetime(df3['month'], format="%Y-%m")
df3['lease_commence_date'] = pd.to_datetime(df3['lease_commence_date'], format="%Y")
df4['month'] = pd.to_datetime(df4['month'], format="%Y-%m")
df4['lease_commence_date'] = pd.to_datetime(df4['lease_commence_date'], format="%Y")

In [ ]:
# Get the new column
df1['remaining_lease'] = 99 - (df1['month'].dt.year - df1['lease_commence_date'].dt.year)
df2['remaining_lease'] = 99 - (df2['month'].dt.year - df2['lease_commence_date'].dt.year)
df3['remaining_lease'] = 99 - (df3['month'].dt.year - df3['lease_commence_date'].dt.year)

In [ ]:
# View the new column
df3.head()

Let's merge the dataframes. Can you figure the function from pandas that can do this? You can refer to the pandas documentation [here](https://pandas.pydata.org/pandas-docs/stable/).

In [ ]:
## To do
## Write a line of code to merge the dataframes


**Solution:**

In [ ]:
# Solution
df = pd.concat([df1,df2,df3,df4], sort=True)
df.tail()

Sort the dataframe in ascending order of dates when the flats were sold.

In [ ]:
# Let's sort the dataframe by month column in ascending order
df = df.sort_values(['month'])
df.head()

#### Some other minor cleaning of the data
After merging data, it is always a good practice to check columns (especially string/character type columns) to make sure the values are consistent. Here, check the "flat_model", "flat_type", "storey_range" and "town" columns for any inconsistent values and correct them.

In [ ]:
# Flat_model column
values = list(set(df['flat_model'].values.tolist()))
values

In [ ]:
df['flat_model'] = df.flat_model.str.lower()
# Standardize repeated values of Multi-Generation and Premium Apartment
df['flat_model'] = df['flat_model'].replace(['multi generation','premium apartment.'], ['multi-generation','premium apartment'])
values = list(set(df['flat_model'].values.tolist()))
values

In [ ]:
# Flat_type column
values = list(set(df['flat_type'].values.tolist()))
values

"flat_type" has variables that are actually the same one variable but are represented differently. Can you correct them?

In [ ]:
## To do
## Write the code to correct the variable values
## HINT: Refer to few cells above



**Solution:**

In [ ]:
# Solution
df['flat_type'] = df['flat_type'].replace(['MULTI GENERATION'], ['MULTI-GENERATION'])
values = list(set(df['flat_type'].values.tolist()))
values 

In [ ]:
# storey_range column
values = list(set(df['storey_range'].values.tolist()))
values

In [ ]:
# town column
values = list(set(df['town'].values.tolist()))
values

#### Create new column from existing columns
The address of each flat is split between columns - block number is in "block" column and street name is in "street_name" column. For ease, it would be preferable to combine these 2 columns into 1 and drop the other 2. Later, this column will be used to find the corresponding postal codes which is missing from the data.

In [ ]:
# Combine block and street name column to form a new column, address. Drop the block and street name columns
df['Address'] = (df['block'] + ' ' + df['street_name']).str.lower()
df = df.drop(['block','street_name'], axis = 1)
print(df.head())
print("Total number of rows in dataset: {}".format(len(df)))

#### Create Postal Code column
Now, from the newly created Address column, the postal codes can be scraped from OneMap APIs by passing the address values to the site and extracting the postal code values from the JSON (JavaScript Object Notation) responses returned.

First, we need to find the unique list of addresses for searching. The dataset contains repeated rows of same addresses as the flats and the blocks could appear in multiple transactions over the years. Getting the unique values reduces the number of API calls needed and avoids redundancy. 

In [ ]:
# Need to get postal code for addresses in dataset
address_list = list(set(df.Address.values.tolist()))
print("Number of unique addresses: {}".format(len(address_list)))

Once the unique addresses are obtained, call the OneMap API sequentially to search the database for matching address and return the JSON response. If there is a valid response returned, check to see that the JSON data is not empty. If empty, indicate a None response for postal code. Otherwise, extract the postal code and push it into a list. 

In [ ]:
search_url = "https://developers.onemap.sg/commonapi/search"

postal_code_list = [] # To store postal codes returned
latitude_list = []
longitude_list = []

In [ ]:
for address in tqdm.tqdm_notebook(address_list):
    query = {'searchVal': address,
             'returnGeom': 'Y',
             'getAddrDetails' : 'Y'
            }
    
    # If an okay response is not returned, keep retrying the API call until it is sucessful. If successful,
    # store the returned postal code and exit the loop. Move to the next address in the list.
    status_check = 0
    while status_check != 200:
        response = requests.get(search_url, params=query)
        if response.status_code == 200:
            data = response.json()
            if (len(data['results'])>0) and (str(data['results'][0]['POSTAL']) != 'NIL'):
                postal_code_list.append(str(data['results'][0]['POSTAL']))
                latitude_list.append(data['results'][0]['LATITUDE'])
                longitude_list.append(data['results'][0]['LONGITUDE'])
            else:
                postal_code_list.append(None) # No postal code returned
                latitude_list.append(None) # No latitude returned
                longitude_list.append(None) # No longitude returned
            status_check = 200
        else:
            print("API Call unsuccessful. Retrying...")

Create a new dataframe of address and postal codes.

In [ ]:
df_add_postal = pd.DataFrame(
    {'Address': address_list,
     'Postal Code': postal_code_list,
     'Latitude': latitude_list,
     'Longitude' : longitude_list
     }
)

Merge the above dataframe into the master dataframe to create the column of postal codes.

In [ ]:
# Merge postal codes to main dataframe
df.set_index('Address', inplace=True)
df_add_postal.set_index('Address', inplace=True)
df = df.join(df_add_postal, how='left').reset_index()

In [ ]:
df.head(15)

### Dropping rows with missing values
The API calls returned some empty postal code values for some addresses. The reasons for the NIL return could be due to the flats/blocks no longer existing i.e. torn down, redeveloped, renamed etc. or the database of OneMap being not up-to-date. It is best to remove these rows of data from the dataset.

In [ ]:
print("Total number of rows in dataset before removing rows with NA, NULL or None values: {}".format(len(df)))
df = df.dropna()
print("Total number of rows in dataset after removing rows with NA, NULL or None values: {}".format(len(df)))

In [ ]:
df.head(25)

### Save the dataset with postal codes

In [ ]:
df.to_csv(DATA_DIR+'HDB_Resale_Data_cleaned.csv', index=False)

## Load MRT Station Locations into Pandas Dataframe (for Tableau Visualization)

The locations of MRT stations around Singapore can be downloaded from [here](https://www.mytransport.sg/content/dam/datamall/datasets/Geospatial/TrainStation.zip). Inside the folder, there are multiple files but of importance, there are 2 - the files that end with *.shp* and *.dbf* extensions. Copy both files out into the DATA_DIR folder after unzipping. The next 2 cells does some manipulation of the data so that it is readable in a Pandas Dataframe.

In [ ]:
import shapefile #the pyshp module

#read file, parse out the records and shapes
mrt_shapefile_path = DATA_DIR + 'MRTLRTStnPtt.shp'
mrt_sf = shapefile.Reader(mrt_shapefile_path)

In [ ]:
#grab the shapefile's field names (omit the first psuedo field)
fields = [x[0] for x in mrt_sf.fields][1:]
records = mrt_sf.records()
x = [s.points[0][0] for s in mrt_sf.shapes()]
y = [s.points[0][1] for s in mrt_sf.shapes()]

#write the records into a dataframe
mrt_df = pd.DataFrame(columns=fields, data=records)

#add the coordinates data to columns called "coords.x" and "coords.y"
mrt_df = mrt_df.assign(coords_x=x,coords_y=y)
mrt_df.head(10)

Now, the location coordinates of the station are in SVY21 format (which is the typical location cooordinates for Singapore addresses). In order to make it comparable to the cooordinates system of the HDB flat addresses found earlier, we need to do some conversion using OneMap APIs.

In [ ]:
convert_url = "https://developers.onemap.sg/commonapi/convert/3414to4326"

latitude_list = []
longitude_list = []

Using the example a few cells above, can you recreate the *for* loop to call the API to get the latitudes and longitudes? HINT: You can zip *mrt_df['coords_x']* and *mrt_df['coords_y']* inside the for iterator as *zip(mrt_df['coords_x'],mrt_df['coords_y'])*. You can also look at the OneMap API documentation [here](https://docs.onemap.sg/#3414-svy21-to-4326-wgs84) to see what are the query parameters needed.

In [ ]:
## To do
## Write a for loop to call the API to get the latitudes and longitudes
## You can copy the for loop from the previous API call cell and modify accordingly


In [ ]:
# Solution
with tqdm.tqdm(total=len(mrt_df)) as pbar:
    for x,y in zip(mrt_df['coords_x'],mrt_df['coords_y']):
        query = {'X': x,
                 'Y': y
                }

        # If an okay response is not returned, keep retrying the API call until it is sucessful. If successful,
        # store the returned coordinates and exit the loop. Move to the next address in the list.
        status_check = 0
        while status_check != 200:
            response = requests.get(convert_url, params=query)
            if response.status_code == 200:
                data = response.json()
                if len(data)>0:
                    latitude_list.append(data['latitude'])
                    longitude_list.append(data['longitude'])
                else:
                    latitude_list.append(None) # No latitude returned
                    longitude_list.append(None) # No longitude returned
                status_check = 200
                pbar.update(1)
            else:
                print("API Call unsuccessful. Retrying...")

Add the latitude and longitudes of the MRT stations found to the dataframe and drop the columns with the SVY21 coordinates.

In [ ]:
mrt_df['latitude'] =  latitude_list
mrt_df['longitude'] =  longitude_list
mrt_df = mrt_df.drop(['OBJECTID','coords_x','coords_y'],axis=1)
mrt_df.head()

In [ ]:
mrt_df.to_csv(DATA_DIR+'MRT_Locations.csv', index=False) # Saving the mrt stations location data to a csv file

## Find nearest MRT station to each address in HDB resale transaction data

The HDB resale data contains 781 030 rows of data and the mrt locations data contains 183 rows of data. Now, we want to find what is the nearest MRT station to each address in the resale data. By brute force, for each address, we need to compute 183 distances and then select the minimum. For the entire dataset, this requires 781 030 x 183 = 142,928,490 operations! This is a very time-consuming process and may take almost a day to run. Here, we present a trick using a data structure known as trees to speed up and simplify the search operations. Specifically, we are using a __vantage-point (VP) tree__ to find the nearest MRT station.
![Vantage Point](images/VP.png)

A VP tree is a metric tree that segregates data in a metric space by choosing a position in the space ("vantage point") and partioning the data points into 2 parts: those points that are nearer to the vantage point than a threshold and those points that are not. By recursively applying this procedure to partition the data into smaller and smaller sets, a tree data structure is created where neighbours in the tree are likely to be neighbours in space. Here, the distance metric used is the __geodesic distance__ which is the shortest distance on the surface of an ellipsoidal model of the earth. This distance computation is provided by the *geopy* package. Once the tree is computed, the nearest lat-lon to the lat-lon of interest is extracted by finding the closest neighbour and then, the corresponding mrt station name is extracted and tagged to the data.

In [ ]:
import vptree
from geopy import distance

In [ ]:
def gps_distance(p1,p2):
    return distance.distance((p1[0],p1[1]),(p2[0],p2[1])).km

In [ ]:
def find_nearest_point(row):
    point = tree.get_nearest_neighbor(row[['Latitude','Longitude']])
    return [point[0],point[1][0],point[1][1]]

In [ ]:
# Build vantage-point(VP) tree in O(n log n) time complexity
tree = vptree.VPTree(mrt_df[['latitude','longitude']].values,gps_distance)

Extract the latitude-longitude information of the HDB flats from the data. Now, there are a lot of duplicate entries which can be removed. This reduces the number of search operations needed.

In [ ]:
df_lat_lon = df[['Latitude','Longitude']]
print("Number of lat-lon (include repeats): {}".format(len(df_lat_lon)))
df_lat_lon = df_lat_lon.drop_duplicates()
print("Number of unique lat-lon: {}".format(len(df_lat_lon)))

Just for comparison sake: Creating a VP-tree is a *O(n log n)* operation where n here is 183. Therefore, creating a tree takes about 414 operations. Searching one point is a *O(log n)* operation which is about 2 operations. So for 8754 data points, about 17 508 operations. So, total number of operations:  about 17 508 vs 142,928,490 operations by brute force method.

In [ ]:
# Find the nearest MRT location in data using the VP tree
df_lat_lon[['Distance to MRT','MRT_Latitude','MRT_Longitude']] = df_lat_lon.apply(find_nearest_point,axis=1,result_type = 'expand')
df_lat_lon = pd.merge(df_lat_lon,mrt_df,left_on=['MRT_Latitude','MRT_Longitude'],right_on=['latitude','longitude'],how='left').drop(['latitude','longitude'],axis=1)
df_lat_lon.head()

In [ ]:
# Finally merge the locations into the main data
df = pd.merge(df, df_lat_lon, on = ['Latitude','Longitude'], how='left')
df.head()

### Save the dataset for Tableau visualization
Now that the dataset with MRT stations has been processed, save the dataset for use later for further analysis.

In [ ]:
df.to_csv(DATA_DIR+'HDB_Resale_Data_mrt_cleaned.csv', index=False)